# Data generation process

This notebook demonstrates the process of generating some synthetic data for causal inference problems.

The diagram shows a list of key points:
- Current salary is the outcome we are interested in.
- Taking a course is believed to have an impact on someone's current salary.
- Current salary is also associated with three factors
  - Budget of the role
  - Salary of the previous job
  - Market rate

**Formal representation**
- $ Y = TE * T + a * X_1 + b * X_2 + c * X_3$
- $ TE = d * X_2^2$
- $T = \begin{cases} 1, & \text{if } X_2 > 50 \\ 0, & \text{if } X_2 \leq 50 \\ \end{cases}$


```mermaid
%%{init: {'theme':'default'}}%%

flowchart
subgraph  
direction BT
x1((X1 \n role budget))
x2((X2 \n previous salary))
x3((X3 \n market rate))
y((Y \n current salary))
t((T \n took a course))
x1 --a--> y
t --TE--> y
x2 --> t
x2 --b--> y
x3 --c--> y
end
```

In [ ]:
import numpy as np